In [99]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [53]:
uof_page = requests.get("https://sbcountyda.org/categories/news-releases/use-of-force-reviews/page/1").content
soup = BeautifulSoup(uof_page)

In [102]:
# TODO : Wrap the request in a try/catch
def _get_uof_catalog_pages():
    """
    Helper method to get use of force catalog pages from San Bernardino site. Fetches base page, calculates number of pages based on results param located at top of page under "<numeric> results"
    """
    # Get the number of pages from the "post-results field on the "Category: Use of Force Reviews" page
    uof_page = requests.get(
        "https://sbcountyda.org/categories/news-releases/use-of-force-reviews/"
    ).content
    soup = BeautifulSoup(uof_page)
    number_of_pages = (
        int(
            re.search(
                "\d*", soup.find_all("div", {"class": "post-results"})[0].text
            ).group()
        )
        % 10
    )
    base_pages = [
        "https://sbcountyda.org/categories/news-releases/use-of-force-reviews/page/{}/".format(
            i
        )
        for i in range(1, number_of_pages + 1)
    ]
    return base_pages


# TODO : Wrap the request in a try/catch
def _get_uof_review_links():
    """
    For each instance of UOF documented by the court, there is an individual page stored on the SB website. This method gives those links, which will then have a link to a pdf of the court doc and police report.
    """
    # Get links to each individiual uof case filing page
    catalog_pages = _get_uof_catalog_pages()
    # For each catalog page, we want to grab the set of individual links on each.
    for i in catalog_pages:
        ind_uof_page = requests.get(i).content
        soup = BeautifulSoup(ind_uof_page)
        for i in soup.find_all("h2", {"class": "entry-title bolt-highlight-font"}):
            regex_match = re.search(r"\<a href=.* rel", str(i)).group()
            # Strip head and tail of match to get just the link. There's probably a better way to search this with bs4
            link_cleaned = (
                str(regex_match).replace('<a href="', "").replace('" rel', "")
            )
        # Courtesy sleep timer
        time.sleep(1)


# TODO : We need to write a function that can grab the .pdf's from each individual use of force case page. Not all pages will have the same format for .pdfs, so we need to search/grep them

https://sbcountyda.org/2023/01/10/officer-involved-shooting-fatal-sugarloaf-brandon-r/
https://sbcountyda.org/2022/12/27/officer-involved-shooting-fatal-san-bernardino-green-c/
https://sbcountyda.org/2022/12/22/officer-involved-shooting-fatal-chino-wang-l/
https://sbcountyda.org/2022/12/13/officer-involved-shooting-fatal-ontario-benavente-j/
https://sbcountyda.org/2022/11/29/officer-involved-shooting-fatal-san-bernardino-rascon-b/
https://sbcountyda.org/2022/11/15/officer-involved-shooting-fatal-barstow-powell-l/
https://sbcountyda.org/2022/10/18/officer-involved-shooting-fatal-san-bernardino-segura-n/
https://sbcountyda.org/2022/10/18/officer-involved-shooting-fatal-victorville-francois-b/
https://sbcountyda.org/2022/10/04/officer-involved-shooting-fatal-fontana-chavira-f/
https://sbcountyda.org/2022/09/20/officer-involved-shooting-fatal-redlands-castillo-j/
https://sbcountyda.org/2022/09/06/officer-involved-shooting-fatal-barstow-latham-c/
https://sbcountyda.org/2022/08/23/officer-in